In [ ]:
# !pip install gradio

In [1]:
import gradio as gr

In [2]:
def combined_pipeline(image):
    try:
        # Step 1: Crop the detected object using Roboflow
        cropped_image = crop_detected_object(image)

        if isinstance(cropped_image, str):  # Check if the result is an error message
            return cropped_image

        # Step 2: Run Florence-2 inference on the cropped image
        extracted_text = run_inference(cropped_image)

        # Step 3: Extract book details from the text using ChatGPT
        details = extract_details(extracted_text)

        return details
    except Exception as e:
        return f"Error in combined_pipeline: {str(e)}"

In [3]:
interface = gr.Interface(
    fn=combined_pipeline,
    inputs=gr.Image(type="pil", mirror_webcam=False),
    outputs=gr.Textbox(),
    title="Object Detection and Image Inference with Book Details Extraction",
    description="Upload an image to detect objects using Roboflow, crop the detected object, run Florence-2 inference, and extract book details using GPT-4o."
)

In [4]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://2e3137e21cf95b07c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [5]:
interface.close()

Closing server running on port: 7860


In [14]:
import numpy as np
import gradio as gr

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img


In [15]:
demo = gr.Interface(sepia, gr.Image(), "image")

In [16]:

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://695c911b2e09f60092.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/ibnu/miniconda3/envs/py312/lib/python3.12/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ibnu/miniconda3/envs/py312/lib/python3.12/site-packages/gradio/route_utils.py", line 321, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ibnu/miniconda3/envs/py312/lib/python3.12/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ibnu/miniconda3/envs/py312/lib/python3.12/site-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ibnu/miniconda3/envs/py312/lib/python3.12/site-packages/anyio/to_thr

In [17]:
demo.close()

Closing server running on port: 7862
